In [1]:
import pickle
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
import copy

In [2]:
np.random.seed(42)
teams = ['BKN', 'MIL', 'GSW', 'LAL', 'IND', 'CHA', 'CHI', 'DET',
          'WAS', 'TOR', 'BOS', 'NYK', 'CLE', 'MEM', 'PHI', 'NOP',
          'HOU','MIN', 'ORL', 'SAS', 'OKC', 'UTA', 'SAC', 'POR',
          'DEN', 'PHX', 'DAL', 'ATL', 'MIA', 'LAC']
teams.reverse()
window = 4 # 4 samples per game (one sample / quarter)

In [3]:
N_players = 5
players = [f'player{i}' for i in range(1, N_players+1)]
n_components = 3 # num of hidden states
n_features = 3 # num of observed states
O_symbols = [0, 1, 2] # under-, avg-, over- performance
H_symbols = [0, 1, 2] # corresponding mental states
T = 100
learning_iterations = 100
initial_dist = np.array([0, 1, 0])

In [4]:
# === M ===
avg_transO = np.array([[0.5, 0.3, 0.2],
                       [0.25, 0.5, 0.25],
                       [0.2, 0.3, 0.5]])

star_transO = np.array([[0.7, 0.3, 0],
                        [0.1, 0.8, 0.1],
                        [0, 0.3, 0.7]])
# === N ===
avg_transH = np.array([[0.8, 0.2, 0.0],
                       [0.1, 0.8, 0.1],
                       [0.0, 0.2, 0.8]])

# === emission_prob ===
avg_emission = np.array([[0.7, 0.3, 0],
                       [0.1, 0.8, 0.1],
                       [0.0, 0.3, 0.7]])

# ===  R ===
R_singleH = np.array([1] + [0] * len(players))
def R_singleHO(player):
    i = int(player[-1])
    arr = [0] * (len(players)+1)
    arr[0] = 0.7
    arr[i] = 0.3
    return np.array(arr)

def R_singleO(player):
    i = int(player[-1])
    arr = [0] * (len(players)+1)
    arr[i] = 1
    return np.array(arr)

def R_star(player, star):
    if player == star:
        return R_singleH
    arr = [0] * (len(players)+1)
    i = int(star[-1])
    arr[i] = 1
    return np.array(arr)
    
R_uniform = np.array([1/(1 + len(players))] * (len(players) + 1))

In [5]:
def init_trans():
    trans_mat = np.zeros((n_components, n_components))
    
    trans_mat[0][0] = np.random.uniform(0.5, 1)
    trans_mat[0][1] = 1 - trans_mat[0][0]
    trans_mat[0][2] = 0.0
    
    trans_mat[1][1] = np.random.uniform(0.5, 1)
    trans_mat[1][0] = (1 - trans_mat[1][1]) / 2
    trans_mat[1][2] = (1 - trans_mat[1][1]) / 2
    
    trans_mat[2][2] = np.random.uniform(0.5, 1)
    trans_mat[2][0] = 0.0
    trans_mat[2][1] = 1 - trans_mat[2][2]
    
    return trans_mat

def init_random():
    mat = np.random.random((n_features, n_components))
    for i in range(n_features):
        mat[i] = mat[i] / np.sum(mat[i])
    return mat

def init_R():
    R = dict()
    for p in players:
        # populate R[p] with random numbers
        R[p] =  np.random.rand(N_players + 1)
        
        # randomly select entries and set them to 0
        nums = np.random.choice(range(N_players + 1), np.random.choice(range(N_players)), replace=False)
        for i in nums:
            R[p][i] = 0
            
        # normalize
        R[p] = R[p] / np.sum(R[p])
    
    return R

In [6]:
def cond(player, h1, h2, t, M_, N_, R_): # P(H_t^player = h1 | H_{t-1}^player = h2, O_{t-1})    
    # Requires M_, N_, R_, Os
    v = [N_[h2][h1]]
    for teammate in players:
        v.append(M_[Os[teammate][t-1]][h1])
            
    v = np.array(v)
    return np.dot(R_[player], v)  

In [7]:
def learn_R(M, N, samples, O):
    R = {p: cp.Variable(len(players) + 1, nonneg=True) for p in players}

    objective = 0
    for H in samples:
        for p in players:
            for t in range(1, T):
                h_t = H[p][t] # state of player p at time t
                v_t = [N[H[p][t-1]][h_t]]
                for teammate in players:
                    v_t.append(M[O[teammate][t-1]][h_t])
                v_t = np.array(v_t)
                prod = R[p] @ v_t
                objective -= cp.log(prod)

    R_constraints = [cp.sum(R[p]) == 1 for p in R]
    prob = cp.Problem(cp.Minimize(objective), R_constraints)
    prob.solve(solver=cp.MOSEK)
    
    R_optimized = {p: R[p].value for p in players}
    
    return R_optimized, prob.value

In [8]:
def learn_M_N(R, samples, O):
    M = cp.Variable((n_features, n_components), nonneg=True)
    N = cp.Variable((n_components, n_components), nonneg=True) 
    E = cp.Variable((n_components, n_features), nonneg=True)
    
    objective = 0
    for H in samples:
        for p in players:
            for t in range(1, T):
                h_t = H[p][t] # state of player p at time t
                v_t = [2*N[H[p][t-1], h_t]]
                for teammate in players:
                    v_t.append(M[O[teammate][t-1], h_t])
                v_t = np.array(v_t)
                prod = R[p] @ v_t
                objective -= cp.log(prod)
    
    M_constraints = [cp.sum(M[i, :]) == 1 for i in range(n_features)]
    N_constraints = [cp.sum(N[i, :]) == 1 for i in range(n_components)]
    
    constraints = M_constraints + N_constraints
    prob = cp.Problem(cp.Minimize(objective), constraints)
    prob.solve(solver=cp.MOSEK)
    
    M_optimized = M.value
    N_optimized = N.value
    
    return M_optimized, N_optimized, prob.value

In [9]:
def learn_E(samples, O):
    E = cp.Variable((n_components, n_features), nonneg=True)
    
    objective = 0
    for H in samples:
        for p in players:     
            for t in range(T):
                objective -= cp.log(E[H[p][t]][O[p][t]])
                
    E_constraints = [cp.sum(E[i, :]) == 1 for i in range(n_components)]
    prob = cp.Problem(cp.Minimize(objective), E_constraints)
    prob.solve(solver=cp.MOSEK)
    E_optimized = E.value
    
    return E_optimized, prob.value

In [10]:
def calculate_forward(M_, N_, E_, R_):
    alpha = {p: [] for p in players}
    alpha_help = {p: [] for p in players}
    
    # Initialize forward parameters
    for p in players:
        alpha_help[p].append(initial_dist)
    for p in players:
        arr = np.array([E_[h][Os[p][0]] * alpha_help[p][0][h] for h in H_symbols])
        arr /= np.sum(arr)
        alpha[p].append(arr)

    # Compute forward parameters (bottom-up)
    for p in players:
        for t in range(1, T):
            if t % 4 == 0:
                alpha_help[p].append(initial_dist)
                alpha[p].append(initial_dist)
                continue
                
            arr = [sum([cond(p, h, h_, t, M_, N_, R_) * alpha[p][t-1][h_] for h_ in H_symbols]) for h in H_symbols]
            arr = np.array(arr)
            alpha_help[p].append(arr)
            
            arr = [E_[h][Os[p][t]] * alpha_help[p][t][h] for h in H_symbols]
            arr = np.array(arr)
            arr /= np.sum(arr)
            alpha[p].append(arr)
            
    return alpha, alpha_help

In [11]:
def E_step(M_, N_, E_, R_, num_of_samples = 10):
    alpha, alpha_help = calculate_forward(M_, N_, E_, R_)
    
    # Sample hidden states from the posterior distribution
    samples = []
    for _ in range(num_of_samples):
        Hs_ = {p: [1] for p in players}

        for p in players:
            for t in range(1, T):
                dist = np.array([alpha[p][t][h] * cond(p, h, Hs_[p][t-1], t, M_, N_, R_) / alpha_help[p][t][h] if alpha_help[p][t][h] != 0 else 0 for h in H_symbols])
                dist /= np.sum(dist)
                Hs_[p].append(np.random.choice(H_symbols, p=dist))
        for p in players:
            Hs_[p] = np.array(Hs_[p])
        samples.append(copy.deepcopy(Hs_))
    
    return samples

def M_step(samples, M, N, R, E, iterations=1):
    M_opt = M
    N_opt = N
    R_opt = copy.deepcopy(R)
    E_opt = E
    val = 1e+100
    for _ in range(iterations):
        try:
#             R_opt, val1 = learn_R(M_opt, N_opt, samples, Os) # Fix M, N and maximize R, E
#             M_opt, N_opt, val1 = learn_M_N(R_opt, samples, Os) # Fix R and maximize M, N, E

            M_opt, N_opt, val1 = learn_M_N(R_opt, samples, Os) # Fix R and maximize M, N, E
            R_opt, val1 = learn_R(M_opt, N_opt, samples, Os) # Fix M, N and maximize R, E
            E_opt, val2 = learn_E(samples, Os)
            
            val = (val1 + val2) / len(samples)
        except cp.error.SolverError as e:
            print(e)
            break
            
    return M_opt, N_opt, R_opt, E_opt, val

def EM(params, iterations = 30, reltol = 1e-3, inctol = 2):
    # Initialize parameters
    M_, N_, E_, R_ = params['M'], params['N'], params['E'], params['R']
    
    cntObjIncreasing = 0
    val = 1e+100 # best objective value achieved so far
    M_opt = M_
    N_opt = N_
    E_opt = E_
    R_opt = copy.deepcopy(R_)
    for i in range(iterations):
        print(f'=== EM iteration {i+1} ===')
        # E-step
        print(f'E-step...')
        samples = E_step(M_, N_, E_, R_)
        
        
        # M-step
        print(f'M-step...')
        M_, N_, R_, E_, obj_val = M_step(samples, M_, N_, R_, E_)
        
            
        print(f'\tObjVal = {obj_val}')
        # Update current best parameters
        if obj_val < val:
            M_opt = M_
            N_opt = N_
            E_opt = E_
            R_opt = copy.deepcopy(R_)
        
        # Stopping conditions
        if abs(val - obj_val) / val < reltol and obj_val < val:
            break
        
        cntObjIncreasing = 0 if obj_val <= val else cntObjIncreasing + 1
        if cntObjIncreasing > inctol:
            break
            
        # Update best objective value so far    
        val = min(val, obj_val)
        
    return M_opt, N_opt, R_opt, E_opt, val

In [12]:
def EM_helper(param_list):
    i = 1
    val_opt = 1e+100
    for params in param_list:
        print(f'==== EM run {i} ====')
        i += 1
        M_, N_, R_, E_, val = EM(params)
        
        if val < val_opt:
            val_opt = val
            M_opt = M_
            N_opt = N_
            E_opt = E_
            R_opt = R_
            
    return M_opt, N_opt, R_opt, E_opt, val_opt

In [13]:
def print_params(M_, N_, E_, R_):
    print(f'M = {np.round(M_, 2)}')
    print(f'N = {np.round(N_, 2)}')
    print(f'E = {np.round(E_, 2)}')
    for p in players:
        print(f'R[{p}] = {np.round(R_[p], 2)}')

In [14]:
def likelihood(params): # Calculate log P(O) (given the model parameters)
    M, N, E, R = params['M'], params['N'], params['E'], params['R']

    # Caluclate forward parameters
    alpha, alpha_help = calculate_forward(M, N, E, R)
    obj = 0
    for p in players:
        for t in range(T):
            if isinstance(E, dict):
                obj += -np.log(sum([E[p][h][Os[p][t]] * alpha_help[p][t][h] for h in H_symbols])) # Pr(O_t^i | O_{1:t-1})
            else:
                obj += -np.log(sum([E[h][Os[p][t]] * alpha_help[p][t][h] for h in H_symbols])) # Pr(O_t^i | O_{1:t-1})

    return obj

def likelihood_test(params_list):
    # Given a list of model parameters returns the model parameters that achieve the highest likelihood
    val_opt = 1e+100
    for params in params_list:
        val = likelihood(params)
        if val < val_opt:
            val_opt = val
            params_opt = params
    return params_opt

In [15]:
for team in teams:
    try:
        print(f'Optimizing team = {team}')
        with open(f'../team-data/observations/{team}_observations.pickle', 'rb') as file:
                observations = pickle.load(file)

        k = list(observations.keys())[0]
        players = list(observations[k].keys())

        Os = {p: [] for p in players}
        for game_id in observations:
            game_stats = observations[game_id]
            for p in players:
                Os[p].extend(game_stats[p])

        with open(f'./profiles2/{team}_profile.pkl', 'rb') as file:
            params = pickle.load(file)

        print(f"Initial Likelihood = {likelihood(params)}")
        print(params)
        M_, N_, R_, E_, val = EM(params, 30)
        print_params(M_, N_, E_, R_)

        print(f"Final Likelihood = {likelihood({'M': M_, 'N': N_, 'E': E_, 'R': R_})}")

        params = dict()
        params['M'] = M_
        params['N'] = N_
        params['E'] = E_
        params['R'] = R_
        with open(f'./results2/{team}_profile.pkl', 'wb') as file:
            pickle.dump(params, file)
            
    except Exception as e:
        print(f'Error for team {team}')
        print(e)

Optimizing team = LAC
Initial Likelihood = 518.2777717875646
{'M': array([[0. , 0.3, 0.7],
       [0.2, 0.6, 0.2],
       [0.7, 0.3, 0. ]]), 'N': array([[0.8, 0.2, 0. ],
       [0.1, 0.8, 0.1],
       [0. , 0.2, 0.8]]), 'E': array([[0.7, 0.2, 0.1],
       [0.1, 0.8, 0.1],
       [0.1, 0.2, 0.7]]), 'R': {'Eric Bledsoe': array([0.8, 0.2, 0. , 0. , 0. , 0. ]), 'Reggie Jackson': array([0.8, 0. , 0.2, 0. , 0. , 0. ]), 'Terance Mann': array([0.8, 0. , 0. , 0.2, 0. , 0. ]), 'Marcus Morris': array([0.8, 0. , 0. , 0. , 0.2, 0. ]), 'Ivica Zubac': array([0.8, 0. , 0. , 0. , 0. , 0.2])}, 'name': 'synthetic-2261'}
=== EM iteration 1 ===
E-step...
M-step...


/usr3/graduate/nikolaou/.local/lib/python3.10/site-packages/cvxpy/problems/problem.py:157: UserWarning: Objective contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn("Objective contains too many subexpressions. "


	ObjVal = 796.6864498663007
=== EM iteration 2 ===
E-step...
M-step...
	ObjVal = 750.1598714117393
=== EM iteration 3 ===
E-step...
M-step...
	ObjVal = 711.4529394405619
=== EM iteration 4 ===
E-step...
M-step...
	ObjVal = 691.5318055635893
=== EM iteration 5 ===
E-step...
M-step...
	ObjVal = 679.0022020639193
=== EM iteration 6 ===
E-step...
M-step...
	ObjVal = 663.0700827280352
=== EM iteration 7 ===
E-step...
M-step...
	ObjVal = 658.4202884741306
=== EM iteration 8 ===
E-step...
M-step...
	ObjVal = 649.3563455304857
=== EM iteration 9 ===
E-step...
M-step...
	ObjVal = 643.1581613767437
=== EM iteration 10 ===
E-step...
M-step...
	ObjVal = 634.5880732366116
=== EM iteration 11 ===
E-step...
M-step...
	ObjVal = 631.1789326810933
=== EM iteration 12 ===
E-step...
M-step...
	ObjVal = 630.5697950566855
M = [[1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]]
N = [[0.18 0.79 0.03]
 [0.07 0.9  0.03]
 [0.03 0.87 0.1 ]]
E = [[0.74 0.   0.25]
 [0.21 0.56 0.23]
 [0.08 0.   0.92]]
R[Eric Bledsoe] = [0.19 0.   

	ObjVal = 536.5614898723394
=== EM iteration 29 ===
E-step...
M-step...
	ObjVal = 536.5263181725824
M = [[0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]]
N = [[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]
E = [[1.   0.   0.  ]
 [0.33 0.44 0.23]
 [0.   0.   1.  ]]
R[Luka Doncic] = [1. 0. 0. 0. 0. 0.]
R[Jalen Brunson] = [0.99 0.   0.01 0.   0.   0.  ]
R[Reggie Bullock] = [1. 0. 0. 0. 0. 0.]
R[Dorian Finney-Smith] = [1. 0. 0. 0. 0. 0.]
R[Dwight Powell] = [1. 0. 0. 0. 0. 0.]
Final Likelihood = 531.8392511126676
Optimizing team = PHX
Initial Likelihood = 553.86622226533
{'M': array([[0.8, 0.2, 0. ],
       [0.1, 0.8, 0.1],
       [0. , 0.2, 0.8]]), 'N': array([[0.1, 0.2, 0.7],
       [0.2, 0.6, 0.2],
       [0.7, 0.2, 0.1]]), 'E': array([[0.7, 0.2, 0.1],
       [0.1, 0.8, 0.1],
       [0.1, 0.2, 0.7]]), 'R': {'Chris Paul': array([0.8, 0.2, 0. , 0. , 0. , 0. ]), 'Devin Booker': array([0.8, 0. , 0.2, 0. , 0. , 0. ]), 'Mikal Bridges': array([0.8, 0. , 0. , 0.2, 0. , 0. ]), 'Jae Crowder': array([0.8, 0. , 0. , 0. , 0

	ObjVal = 477.56561388085595
=== EM iteration 2 ===
E-step...
M-step...
	ObjVal = 425.48955729526295
=== EM iteration 3 ===
E-step...
M-step...
	ObjVal = 383.16269159966805
=== EM iteration 4 ===
E-step...
M-step...
	ObjVal = 350.8549489264266
=== EM iteration 5 ===
E-step...
M-step...
	ObjVal = 340.67830164118493
=== EM iteration 6 ===
E-step...
M-step...
	ObjVal = 336.23379723520355
=== EM iteration 7 ===
E-step...
M-step...
	ObjVal = 335.1000917542233
=== EM iteration 8 ===
E-step...
M-step...
	ObjVal = 331.0829222600949
=== EM iteration 9 ===
E-step...
M-step...
	ObjVal = 331.23488848745825
=== EM iteration 10 ===
E-step...
M-step...
	ObjVal = 331.1297710164191
=== EM iteration 11 ===
E-step...
M-step...
	ObjVal = 330.1098818781432
=== EM iteration 12 ===
E-step...
M-step...
	ObjVal = 329.0887774510669
=== EM iteration 13 ===
E-step...
M-step...
	ObjVal = 327.6125227847075
=== EM iteration 14 ===
E-step...
M-step...
	ObjVal = 332.00405466220366
=== EM iteration 15 ===
E-step...
M-s

	ObjVal = 877.2140017785835
=== EM iteration 2 ===
E-step...
M-step...
	ObjVal = 850.4266751869051
=== EM iteration 3 ===
E-step...
M-step...
	ObjVal = 815.902162608127
=== EM iteration 4 ===
E-step...
M-step...
	ObjVal = 782.5566765514628
=== EM iteration 5 ===
E-step...
M-step...
	ObjVal = 759.762658290321
=== EM iteration 6 ===
E-step...
M-step...
	ObjVal = 734.2115030208761
=== EM iteration 7 ===
E-step...
M-step...
	ObjVal = 718.4259033052731
=== EM iteration 8 ===
E-step...
M-step...
	ObjVal = 697.8374203785194
=== EM iteration 9 ===
E-step...
M-step...
	ObjVal = 687.5650027279808
=== EM iteration 10 ===
E-step...
M-step...
	ObjVal = 675.6106292312409
=== EM iteration 11 ===
E-step...
M-step...
	ObjVal = 661.0542302257458
=== EM iteration 12 ===
E-step...
M-step...
	ObjVal = 654.403207920609
=== EM iteration 13 ===
E-step...
M-step...
	ObjVal = 654.5208954541229
=== EM iteration 14 ===
E-step...
M-step...
	ObjVal = 652.6944122000784
=== EM iteration 15 ===
E-step...
M-step...
	Ob

	ObjVal = 780.429436536988
=== EM iteration 2 ===
E-step...
M-step...
	ObjVal = 754.5704615764737
=== EM iteration 3 ===
E-step...
M-step...
	ObjVal = 723.966578545282
=== EM iteration 4 ===
E-step...
M-step...
	ObjVal = 680.5708387022198
=== EM iteration 5 ===
E-step...
M-step...
	ObjVal = 665.9601722580976
=== EM iteration 6 ===
E-step...
M-step...
	ObjVal = 643.4914741551344
=== EM iteration 7 ===
E-step...
M-step...
	ObjVal = 619.1558379258258
=== EM iteration 8 ===
E-step...
M-step...
	ObjVal = 601.6581125028708
=== EM iteration 9 ===
E-step...
M-step...
	ObjVal = 591.3190052979929
=== EM iteration 10 ===
E-step...
M-step...
	ObjVal = 581.5198897894534
=== EM iteration 11 ===
E-step...
M-step...
	ObjVal = 575.4158473034697
=== EM iteration 12 ===
E-step...
M-step...
	ObjVal = 568.2734605143125
=== EM iteration 13 ===
E-step...
M-step...
	ObjVal = 568.5955485853765
=== EM iteration 14 ===
E-step...
M-step...
	ObjVal = 572.4947155814243
=== EM iteration 15 ===
E-step...
M-step...
	O

	ObjVal = 717.7960862212907
=== EM iteration 3 ===
E-step...
M-step...
	ObjVal = 682.9840235625555
=== EM iteration 4 ===
E-step...
M-step...
	ObjVal = 668.4562363562634
=== EM iteration 5 ===
E-step...
M-step...
	ObjVal = 655.8319253930862
=== EM iteration 6 ===
E-step...
M-step...
	ObjVal = 642.181663345724
M = [[0.09 0.   0.91]
 [0.   1.   0.  ]
 [1.   0.   0.  ]]
N = [[0.01 0.92 0.07]
 [0.08 0.82 0.1 ]
 [0.12 0.72 0.16]]
E = [[0.65 0.01 0.35]
 [0.19 0.63 0.18]
 [0.31 0.   0.69]]
R[Marcus Smart] = [0.62 0.09 0.   0.03 0.   0.26]
R[Jaylen Brown] = [0.  0.  0.3 0.  0.  0.7]
R[Jayson Tatum] = [0.6  0.02 0.04 0.05 0.29 0.  ]
R[Grant Williams] = [0.35 0.   0.05 0.03 0.03 0.55]
R[Al Hordford] = [0. 0. 0. 0. 0. 1.]
Final Likelihood = 474.8848991117549
Optimizing team = TOR
Initial Likelihood = 571.1654105261808
{'M': array([[0. , 0.3, 0.7],
       [0.2, 0.6, 0.2],
       [0.7, 0.3, 0. ]]), 'N': array([[0.8, 0.2, 0. ],
       [0.1, 0.8, 0.1],
       [0. , 0.2, 0.8]]), 'E': array([[0.7, 0.2,

	ObjVal = 843.8875372554583
=== EM iteration 2 ===
E-step...
M-step...
	ObjVal = 812.4351295577083
=== EM iteration 3 ===
E-step...
M-step...
	ObjVal = 788.1236722231516
=== EM iteration 4 ===
E-step...
M-step...
	ObjVal = 752.8533725736736
=== EM iteration 5 ===
E-step...
M-step...
	ObjVal = 726.2126958392146
=== EM iteration 6 ===
E-step...
M-step...
	ObjVal = 698.1740608169633
=== EM iteration 7 ===
E-step...
M-step...
	ObjVal = 681.2900677622712
=== EM iteration 8 ===
E-step...
M-step...
	ObjVal = 675.0968874251132
=== EM iteration 9 ===
E-step...
M-step...
	ObjVal = 665.2904552922337
=== EM iteration 10 ===
E-step...
M-step...
	ObjVal = 656.9835446027648
=== EM iteration 11 ===
E-step...
M-step...
	ObjVal = 648.1448585455609
=== EM iteration 12 ===
E-step...
M-step...
	ObjVal = 634.9395273070093
=== EM iteration 16 ===
E-step...
M-step...
	ObjVal = 637.358814817733
M = [[0.   0.   1.  ]
 [1.   0.   0.  ]
 [0.75 0.   0.25]]
N = [[0.06 0.83 0.11]
 [0.01 0.98 0.01]
 [0.04 0.89 0.07]]

M-step...
	ObjVal = 905.4838168562644
=== EM iteration 2 ===
E-step...
M-step...
	ObjVal = 874.9202164343326
=== EM iteration 3 ===
E-step...
M-step...
	ObjVal = 831.1727753906462
=== EM iteration 4 ===
E-step...
M-step...
	ObjVal = 792.2338149011488
=== EM iteration 5 ===
E-step...
M-step...
	ObjVal = 762.9866289336736
=== EM iteration 6 ===
E-step...
M-step...
	ObjVal = 726.3958545603294
=== EM iteration 7 ===
E-step...
M-step...
	ObjVal = 702.4623919401494
=== EM iteration 8 ===
E-step...
M-step...
	ObjVal = 669.8717964415048
=== EM iteration 9 ===
E-step...
M-step...
	ObjVal = 656.4494742418086
=== EM iteration 10 ===
E-step...
M-step...
	ObjVal = 653.8777023783384
=== EM iteration 11 ===
E-step...
M-step...
	ObjVal = 648.9669970808027
=== EM iteration 12 ===
E-step...
M-step...
	ObjVal = 639.3776442454616
=== EM iteration 13 ===
E-step...
M-step...
	ObjVal = 634.062453374816
=== EM iteration 14 ===
E-step...
M-step...
	ObjVal = 632.6020225753934
=== EM iteration 15 ===
E-step...
M

In [16]:
players

['Spencer Dinwiddie',
 'Cam Thomas',
 'Mikal Bridges',
 'Dorian Finney-Smith',
 'Nic Claxton']